In [1]:
import os

In [2]:
%pwd

'd:\\End-to-End_Flash_App\\research'

In [3]:
os.chdir("../")
%pwd

'd:\\End-to-End_Flash_App'

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    max_depth: int
    min_samples_split: int
    random_state: int
    target_column: str

In [5]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [18]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.gini
        schema=self.schema.TARGET_COLUMN
        create_directories([config.root_dir])
        model_trainer_config=ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            max_depth=params.max_depth,
            min_samples_split=params.min_samples_split,
            random_state=params.random_state,
            target_column=schema.name
        )
        return model_trainer_config

In [10]:
import pandas as pd
import os
from src.datascience import logger
from sklearn.tree import DecisionTreeClassifier
import joblib

In [12]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]


        clf = DecisionTreeClassifier(max_depth=self.config.max_depth, min_samples_split=self.config.min_samples_split, random_state=self.config.random_state)
        clf.fit(train_x, train_y)

        joblib.dump(clf, os.path.join(self.config.root_dir, self.config.model_name))

In [19]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2024-12-19 05:00:36,687: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-19 05:00:36,796: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-19 05:00:36,796: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-19 05:00:36,808: INFO: common: created directory at: artifacts]
[2024-12-19 05:00:36,841: INFO: common: created directory at: artifacts/model_trainer]
